### Imports and Configuration

In [3]:
import gymnasium as gym

import time
import numpy as np
import matplotlib.pyplot as plt
import itertools
from tqdm.notebook import tqdm
import pandas as pd
import pickle

In [6]:
configuration = {

    # Parametrization bellow cannot be changed
    "lanes_count" : 10, # The environment must always have 10 lanes
    "vehicles_count": 50, # The environment must always have 50 other vehicles
    "duration": 120,  # [s] The environment must terminate never before 120 seconds
    "other_vehicles_type": "highway_env.vehicle.behavior.IDMVehicle", # This is the policy of the other vehicles
    "initial_spacing": 2, # Initial spacing between vehicles needs to be at most 2

    # Refer to refer to https://highway-env.farama.org/observations/ to change observation space type
    "observation": {
        "type": "Kinematics"
    },

    # Refer to refer to https://highway-env.farama.org/actions/ to change action space type
    "action": {
        "type": "DiscreteMetaAction",
    },

    # Parameterization bellow can be changed (as it refers mostly to the reward system)
    # "collision_reward": -10,  # The reward received when colliding with a vehicle. (Can be changed)
    # "reward_speed_range": [20, 30],  # [m/s] The reward for high speed is mapped linearly from this range to [0, HighwayEnv.HIGH_SPEED_REWARD]. (Can be changed)
    "simulation_frequency": 15, #15,  # [Hz] (Can be changed)
    "policy_frequency": 5, #5,  # [Hz] (Can be changed)

    "collision_reward": -100,  # The reward received when colliding with a vehicle.
    "right_lane_reward": 0.1,  # The reward received when driving on the right-most lanes, linearly mapped to
    # zero for other lanes.
    "high_speed_reward": 100,  # The reward received when driving at full speed, linearly mapped to zero for
    # lower speeds according to config["reward_speed_range"].
    "lane_change_reward": 0,  # The reward received at each lane change action.
    "reward_speed_range": [20, 30],
    
    # Parameters defined bellow are purely for visualiztion purposes! You can alter them as you please
    "screen_width": 800,  # [px]
    "screen_height": 600,  # [px]
    "centering_position": [0.5, 0.5],
    "scaling": 5,
    "show_trajectories": True,
    "render_agent": True,
    "offscreen_rendering": False
}

default_config = configuration.copy()

### Checking the environment 

In [7]:
occupancyGrid = configuration.copy()
occupancyGrid["observation"] =  {
    "type": "OccupancyGrid",
    # "vehicles_count": 50,
    "features": [
                "presence",
                #"x", "y", 
                #"vx", "vy"
                ],
    # "features_range": {
    #      "x": [-500, 500],
    #      "y": [-500, 500],
    #     "vx": [-20, 20],
    #     "vy": [-20, 20]
    # },
    "grid_size": [[-100, 100], [-100, 100]],    # X controls how many lanes, Y controls how far ahead
    "grid_step": [1, 1],
    #"absolute": False,                     # Not implemented in the library
    #"as_image": True,
    # "align_to_vehicle_axes" : True
}

# The higher the number, the more frequent the policy and the simulation frequencies, the slower the simulation
occupancyGrid["simulation_frequency"] = 15
occupancyGrid["policy_frequency"] = 1

env = gym.make('highway-v0', render_mode='human', config=occupancyGrid)

obs, info = env.reset(seed = 30)

env.close()

obs

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [5]:
# Generate an episode of the environment and show the rewards and cumulative rewards
cum_reward = 0
with gym.make("highway-v0", config=occupancyGrid, render_mode='human') as env:
    obs = env.reset()
    for _ in range(1000):
        action = env.action_space.sample()
        obs, reward, done, truncated, info = env.step(action)
        cum_reward += reward
        print(reward, cum_reward)
        if done:
            break

0.7470041889935329 0.7470041889935329
0.7497703427235373 1.4967745317170702
0.7497916630997844 2.2465661948168547
0.747062037436638 2.9936282322534926
0.7498258836702131 3.7434541159237056
0.9570215828047481 4.700475698728454
0.992425312301707 5.692901011030161
0.9984753812783485 6.691376392308509
0.9971407464067447 7.6885171387152536
0.9996186311465904 8.688135769861844
0.9996605856977958 9.687796355559641
0.7924914156140906 10.480287771173732
0.7542418714829565 11.234529642656689
0.747622191656039 11.982151834312727
0.7498691284710792 12.732020962783807
0.7497170063753728 13.48173796915918
0.7468413438148681 14.228579312974048
0.7496048082651992 14.978184121239247
0.7469279147275342 15.72511203596678
5.552779165969887e-05 15.72516756375844


In [44]:
def fix_reward(reward, colision_reward=-1000, skew_speed=1, done=True): 
    """
    This function is used to correct the reward function, which is not correctly outputted by the environment
    Params: 
        reward: float, the reward to fix
        colision_reward: float, the colision reward to fix
        skew_speed: float, the skew speed to fix. It is an exponent applied to the reward
        done: bool, double check if the reward was indeed a colision
    """
    if reward < 0.5 and done:
        return colision_reward
    else:
        return reward**skew_speed

def decode_meta_action(action):
    """
    Function to output the corresponding action in text-form
    """
    assert action in range(5), "The action must be between 0 and 4"
    if action == 0:
        return "LANE_LEFT"
    elif action == 1:
        return "IDLE"
    elif action == 2:
        return "LANE_RIGHT"
    elif action == 3:
        return "FASTER"
    elif action == 4:
        return "SLOWER"


class OccupancyGrid():
    def __init__(
            self,
            grid_size=[[-50, 50], [-50, 50]],  # X controls the lane-width, Y controls how far ahead
            grid_step=[1, 1],
            state_type='lane-wise',
            n_closest=3,
            ss_bins=[5,6],
            crop_dist=[[-10,10], [-10,25]],
            policy=None,
            sim_frequency=20,
            policy_frequency=2,
            render_mode = 'human',
            seed = 50,
    ):
        """
        Occupancy view class constructor
        Arguments:
            grid_size: list of lists, the size of the grid in the x and y direction, where x controls the lane-width and y controls how far ahead. Lanes are 5m wide, and the car position is (0,0)
            grid_step: list, the step size of the grid in the x and y direction, in meters
            state_type: str, the type of state to use. Options are 'lane-wise' or 'n_neighbours'
            n_closest: int, the number of closest cars to consider in the state space
            ss_bins: list, the number of bins to divide the x and y directions
            crop_dist: list of lists, the distance to crop the x and y directions, above which the values will be clipped
            policy: function, the policy to use in the simulation
            sim_frequency: int, the frequency of the simulation
            policy_frequency: int, the frequency of the policy
            render_mode: str, the mode to render the simulation
            seed: int, the seed to use in the simulation
        """

        self.grid_size = grid_size
        self.grid_step = grid_step
        self.config = default_config.copy()
        self.config["observation"] =  {
            "type": "OccupancyGrid",
            "features": ["presence"],
            "grid_size": grid_size,    # X controls how many lanes, Y controls how far ahead
            "grid_step": grid_step,
        }
        self.config["simulation_frequency"] = sim_frequency
        self.config["policy_frequency"] = policy_frequency
        self.render_mode = render_mode
        self.seed = seed
        self.state_type = state_type
        self.policy = policy
        self.n_closest, self.ss_bins, self.crop_dist = n_closest, ss_bins, crop_dist
        self.initialize_states()


    def initialize_states(self):
        """
        Initialize the states of the occupancy grid
        """
        # Start the environment
        with gym.make('highway-v0', render_mode=self.render_mode, config=self.config) as env:
            obs, info = env.reset(seed = self.seed)
            self.current_obs = obs
            self.env = env
        
        if self.state_type == 'lane-wise':
            self.initialize_lane_wise()

        elif self.state_type == 'n_neighbours':
            self.initialize_n_neighbours()
        
    def initialize_n_neighbours(self):
        # States will be stored in a dictionary, with the key being ((x1,x2,...,xn), (y1,y2,...,yn)), and n is the number of neighbors
        # Make ss_bins[0] from the crop_dist[0] and ss_bins[1] from crop_dist[1]
        self.bins_left_right = np.linspace(self.crop_dist[0][0], self.crop_dist[0][1], self.ss_bins[0])
        self.bins_front = np.linspace(self.crop_dist[1][0], self.crop_dist[1][1], self.ss_bins[1])
        self.bins_back = self.bins_front

        # Each of the nearest neighbors will have a state of the form (x,y). Create the first key of the dictionary in the form (x1,x2,...xn)
        x_keys = list(itertools.product(self.bins_left_right, repeat=self.n_closest))
        y_keys = list(itertools.product(self.bins_front, repeat=self.n_closest))
        self.states = list(itertools.product(x_keys, y_keys))

    def initialize_lane_wise(self):
        self.bins_front = [5,10,30]   #  [5,10,15,30]
        self.bins_back = [5,10,30]
        self.bins_left_right = [-12,-5,5,12]   # [-20,-10,-5,5,10,20]
        self.states = list(itertools.product(self.bins_front, self.bins_back, self.bins_left_right, self.bins_left_right))

    def get_car_positions(self):
        """
        Get the car positions in the occupancy grid
        Returns:
            car_positions: np.array, the car positions in the occupancy grid, in the form ([x1,y1], [x2,y2], ...)
        """ 
        positions = np.nonzero(self.current_obs[0])
        car_positions = np.array([positions[0]*self.grid_step[0] + self.grid_size[0][0], positions[1]*self.grid_step[1] + self.grid_size[1][0]]).T
        return car_positions

    def get_n_closest(self):
        """
        Get the n closest cars to the agent
        Returns:
            closest_car_positions: np.array, the positions of the n closest cars to the agent. If there are less than n_closest cars, the array is padded with the crop_dist values
        """

        car_positions = self.get_car_positions()
        distances = np.linalg.norm(car_positions, axis=1)

        # Remove the agent position
        closest = np.argsort(distances)[1:self.n_closest+1]
        closest_car_positions = car_positions[closest]

        # If there are less than n_closest cars, pad the array with the crop_dist values
        if len(closest_car_positions) < self.n_closest:
            n_missing = self.n_closest - len(closest_car_positions)
            closest_car_positions = np.pad(closest_car_positions, ((0, n_missing), (0,0)), 'constant', constant_values=(self.crop_dist[0][0], self.crop_dist[1][0]))

        # Values that are 
        return closest_car_positions
    
    def get_state(self, type='lane-wise', decode=False):
        """
        Get the state of the environment
        Arguments:
            type: str, the type of state to get. Options are 'n_neighbours' or 'lane-wise' :
                n_neighbours: the state is the n closest neighbours in the form (x1,x2,...,xn), (y1,y2,...,yn)
                lane-wise: the state is a matrix with the binned distances of the agent to the car in front, back, left-lane and right-lane
            decode: bool, whether to decode the state
        """
        assert type in ['n_neighbours', 'lane-wise'], "The type of state must be either 'n_neighbours' or 'lane-wise'"
        if type == 'n_neighbours':
            state = self.state_n_neighbours()
        elif type == 'lane-wise':
            state = self.state_lane_wise(decode=decode)
        return state

    def state_n_neighbours(self):
        """
        Get the state of the environment in a neighbour-wise manner
        Returns:
            state: tuple, the state of the environment in the form (x1,x2,...,xn), (y1,y2,...,yn)
        """
        n_closest = self.get_n_closest()
        # For the closest cars, get the state
        state_x, state_y = [], []
        # Get the bin values for each of the x,y positions, and return a tuple with the values
        for car in n_closest:
            x = np.digitize(car[0], self.x_bins) - 1
            y = np.digitize(car[1], self.y_bins) - 1
            x_val, y_val = self.x_bins[x], self.y_bins[y]
            state_x.append(x_val)
            state_y.append(y_val)
        state = (tuple(state_x), tuple(state_y))
        return tuple(state)
    
    def state_lane_wise(self, decode=False):
        """
        Get the state of the environment in a lane-wise manner
        Arguments:
            state: np.array, the state of the environment with the binned distances of the agent to the car in front, back, left-lane and right-lane 
            decode: bool, whether to return the state in a decoded manner
        """
        # Get the car positions in relation to the agent. The agent is at position (0,0)
        car_positions = self.get_car_positions()

        # The same lane cars are the ones with the same x+-2, while front and back are the ones with y > 0 and y < 0, respectively
        same_lane = car_positions[np.abs(car_positions[:,0]) <= 2]
        front = same_lane[(same_lane[:,1] > 0) & (same_lane[:,1] < 30)]
        back = same_lane[(same_lane[:,1] < 0) & (same_lane[:,1] > -30)]

        # The other lane cars have to be in the range of (2,7] and [-2,-7), for the left and right lanes, respectively
        left_lane = car_positions[(car_positions[:,0] < -2) & (car_positions[:,0] >= -7) & (np.abs(car_positions[:,1]) < 20)]
        right_lane = car_positions[(car_positions[:,0] > 2) & (car_positions[:,0] <= 7) & (np.abs(car_positions[:,1]) < 20)]
        
        # Now we need to get the cars that are the closest from the arrays above
        front_dist = np.min(front[:,1]) if len(front) > 0 else 30            # The one with the smallest y value, e.g.: (0, 4) is closer than (0,12)
        back_dist = -np.max(back[:,1]) if len(back) > 0 else 30               # The one with the largest y value, e.g.: (0, -4) is closer than (0,-12)     
        
        # For the left and right lanes, we need to get the closest euclidean distance
        left_closest = left_lane[np.argmin(np.linalg.norm(left_lane, axis=1))] if len(left_lane) > 0 else np.array([30,30])
        right_closest = right_lane[np.argmin(np.linalg.norm(right_lane, axis=1))] if len(right_lane) > 0 else np.array([30,30])
        
        # If the left or right cars are in front of the agent, then the euclidean distance will be positive, otherwise negative
        left_dist = np.linalg.norm(left_closest) if left_closest[1] > 0 else -np.linalg.norm(left_closest)
        right_dist = np.linalg.norm(right_closest) if right_closest[1] > 0 else -np.linalg.norm(right_closest)

        front_dist = self.bins_front[np.digitize(front_dist, self.bins_front)-1]
        back_dist = self.bins_back[np.digitize(back_dist, self.bins_back)-1]
        left_dist = self.bins_left_right[np.digitize(left_dist, self.bins_left_right)-1]
        right_dist = self.bins_left_right[np.digitize(right_dist, self.bins_left_right)-1]

        state = tuple((front_dist, back_dist, left_dist, right_dist)) if not decode else "Front: {}, Back: {}, Left: {}, Right: {}".format(front_dist, back_dist, left_dist, right_dist)
        return state

    def test_env(self):
        """
        Function to test the environment with a random policy, or with a policy
        """
        obs, info = self.env.reset(seed = self.seed)
        self.current_obs = obs
        done = False
        while not done:
            # start = time.time()
            if self.policy is None:
                action = self.env.action_space.sample()
            else:
                action = self.policy()
            obs, reward, done, truncate, info = self.env.step(action)
            self.current_obs = obs
            print(self.get_state(type='lane-wise', decode=True), decode_meta_action(action), fix_reward(reward, skew_speed=2))
            time.sleep(1)
            # end = time.time()
            # print(f"Time taken: {end-start}")
        self.env.close()
        return info["score"]

# ------------------- SARSA -------------------    
class Sarsa(OccupancyGrid):
    def __init__(
        self,
        alpha=0.75,
        gamma=0.95,
        epsilon=0.6,
        print_stats=False,
        **kwargs,
        ):
        """
        SARSA class constructor
        Arguments:
            alpha: float, the learning rate
            gamma: float, the discount factor
            m: int, the number of episodes to train the agent for
            epsilon: float, the epsilon value for the epsilon-greedy policy
            print_stats: bool, whether to print the statistics during initialization
        """

        super().__init__(**kwargs)
        self.initialize_Q(print_stats)
        self.alpha, self.gamma, self.epsilon = alpha, gamma, epsilon

    def policy_Q(self, state):
        values = [self.Q[(state, action)] for action in range(5)]
        return np.argmax(values)   

    def initialize_Q(self, print_stats = False):
        # Combine the possible states with the possible actions
        keys = list(itertools.product(self.states, range(5)))       # 5 possible actions, 0-4: left, idle, right, accelerate, decelerate
        if len(keys) > 150000:
            print("Warning: The number of states is too large, consider reducing the number of states")
        if print_stats:
            print(f"Number of states: {len(keys)}")   
        self.Q = {key: 0 for key in keys}

    def epsilon_greedy(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(5)
        else:
            values = [self.Q[(state, action)] for action in range(5)]
            return np.argmax(values)
        
    def train(self, m=50, verbose=0): 
        env = gym.make('highway-v0', render_mode=None, config=self.config)
        obs, info = env.reset(seed = self.seed)
        self.current_obs = obs
        done = False
        state = self.get_state()
        action = self.epsilon_greedy(state)
        q_explored = 0
        for i in tqdm(range(m)):
            env.reset()
            cum_reward = 0
            done = False
            while not done:
                next_obs, reward, done, truncate, info = env.step(action)
                reward = fix_reward(reward, done=done)
                next_state = self.get_state()
                next_action = self.epsilon_greedy(next_state)

                print(next_state, decode_meta_action(next_action), reward) if verbose > 2 else None
                if self.Q[(state, action)] == 0:
                    q_explored += 1
                cum_reward += reward

                self.Q[(state, action)] += self.alpha*(reward + self.gamma*self.Q[(next_state, next_action)] - self.Q[(state, action)])
                state, action = next_state, next_action
                self.current_obs = next_obs
            print(f"Episode {i+1} completed, cumulative reward: {cum_reward}") if verbose > 0 else None
            print(f"Q explored: {100*q_explored/len(self.Q)}") if verbose > 1 else None
        env.close()

    def test(self):
        env = gym.make('highway-v0', render_mode=self.render_mode, config=self.config)
        obs, info = env.reset(seed = self.seed)
        self.current_obs = obs
        done = False
        state = self.get_state(self.state_type)
        action = self.policy_Q(state)
        while not done:
            next_obs, reward, done, truncate, info = env.step(action)
            next_state = self.get_state(self.state_type)
            next_action = self.policy_Q(next_state)
            state, action = next_state, next_action
            self.current_obs = next_obs
            print(next_state)
        env.close()
        return info["score"]

- grid_size=[[-50, 50], [-50, 50]],  # X controls the lane-width, Y controls how far ahead
- grid_step=[1, 1],
- n_closest=3,
- ss_bins=[5,6],
- crop_dist=[[-10,10], [-10,25]],
- policy=None,
- sim_frequency=15,
- policy_frequency=5,
- render_mode = 'human',
- seed = 50,

In [45]:
a = Sarsa(print_stats=True, epsilon=0.4)

Number of states: 720


In [46]:
a.train(m=50, verbose=1)

  0%|          | 0/50 [00:00<?, ?it/s]

Episode 1 completed, cumulative reward: -992.5979704320507
Episode 2 completed, cumulative reward: -990.2825113242836
Episode 3 completed, cumulative reward: -990.9409494892124
Episode 4 completed, cumulative reward: -990.5974050429357
Episode 5 completed, cumulative reward: -966.0344252934487
Episode 6 completed, cumulative reward: -987.2215439237193
Episode 7 completed, cumulative reward: -944.4623444411777
Episode 8 completed, cumulative reward: -973.4765640393324
Episode 9 completed, cumulative reward: -979.0561081281122
Episode 10 completed, cumulative reward: -996.1908800502778
Episode 11 completed, cumulative reward: -990.9925324148519
Episode 12 completed, cumulative reward: -983.9203834708939
Episode 13 completed, cumulative reward: -993.3190179108218
Episode 14 completed, cumulative reward: -982.7329815915542
Episode 15 completed, cumulative reward: -988.52928711729
Episode 16 completed, cumulative reward: -987.4232627071336
Episode 17 completed, cumulative reward: -991.16525

In [47]:
count = 0
for keys, values in a.Q.items():
    if values != 0:
        count += 1

print(100*count/len(a.Q))

16.52777777777778


In [48]:
a.test()

(10, 30, 12, 12)
(5, 30, 12, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, 12, 12)
(30, 30, 12, 12)
(30, 30, 12, 12)
(30, 30, 5, 12)
(30, 30, 5, 12)
(30, 30, -5, 12)
(30, 30, -5, 12)
(30, 30, -12, 12)
(30, 30, -12, 12)
(30, 30, -12, 

KeyError: 'score'

In [132]:
new_occupancy = OcupancyGrid(render_mode=None)
# print(new_occupancy.x_bins, new_occupancy.y_bins, len(new_occupancy.states))
# new_occupancy.test_env()

___________________________
### Old stuff

In [203]:
def return_car_positions(obs, grid_step=1, grid_size=50):
    # car_positions = []
    # for i in range(int(grid_size*2 / grid_step)):
    #     for j in range(int(grid_size*2 / grid_step)):
    #         if obs[0,i,j] == 1:
    #             car_positions.append([i*grid_step - grid_size, j*grid_step - grid_size])
    positions = np.nonzero(obs[0])
    car_positions = list(zip(positions[0]*grid_step - grid_size, positions[1]*grid_step - grid_size))
    return car_positions

def get_n_closest(obs, grid_step=1, grid_size=50, n=3):
    car_positions = return_car_positions(obs, grid_step, grid_size)
    distances = [np.linalg.norm(car) for car in car_positions]
    closest = np.argsort(distances)[1:n+1]
    return [car_positions[i] for i in closest]

def get_distances(car_positions):
    return [np.linalg.norm(car) for car in car_positions]

In [45]:
env.action_space.sample()

0

The action space is as follows
</br></br>
ACTIONS_ALL = {
        0: 'LANE_LEFT',
        1: 'IDLE',
        2: 'LANE_RIGHT',
        3: 'FASTER',
        4: 'SLOWER'
    }

In [204]:
# Render the environment slow motion and print the observations
env = gym.make('highway-v0', render_mode='human', config=occupancyGrid)
env.reset(seed = 500)
env.render()
for _ in range(100):
    print(env.action_space.sample())
    obs, reward, done, truncate, info = env.step(env.action_space.sample())
    env.render()
    time.sleep(0.1)
    #print(obs)
    if done:
        break
env.close()

[9.848857801796104, 22.47220505424423]
[10.295630140987, 22.47220505424423]
[10.63014581273465, 21.095023109728988]
[11.313708498984761, 21.095023109728988]
[12.041594578792296, 20.591260281974]
[13.601470508735444, 19.697715603592208]
[15.264337522473747, 18.384776310850235]
[16.1245154965971, 18.027756377319946]
[16.492422502470642, 17.88854381999832]
[16.278820596099706, 18.788294228055936]
[15.132745950421556, 18.788294228055936]
[14.560219778561036, 17.88854381999832]
[14.560219778561036, 17.88854381999832]
[13.601470508735444, 17.88854381999832]
[12.649110640673518, 17.88854381999832]
[11.40175425099138, 18.788294228055936]
[9.486832980505138, 18.788294228055936]
[7.615773105863909, 18.384776310850235]
[5.385164807134504, 18.973665961010276]
[3.605551275463989, 18.439088914585774]
